In [22]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"silpasadhukhan","key":"5397797fe203f427976248d5c72b4e09"}'}

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [23]:
!kaggle datasets download -d tongpython/cat-and-dog


Dataset URL: https://www.kaggle.com/datasets/tongpython/cat-and-dog
License(s): CC0-1.0
cat-and-dog.zip: Skipping, found more recently modified local copy (use --force to force download)


In [24]:
import zipfile

with zipfile.ZipFile("cat-and-dog.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")


In [25]:
import os
print(os.listdir("/content/dataset/training_set"))
print(os.listdir("/content/dataset/test_set"))


['training_set']
['test_set']


# ***import module***

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ***Create Dataframe for input and output***

In [27]:
import os
import cv2
import numpy as np

data_dir = "/content/dataset/training_set/training_set"
img_size = 64  # reduce size for SVM

X = []
y = []

# Iterate through subdirectories (cats and dogs)
for category in os.listdir(data_dir):
    category_path = os.path.join(data_dir, category)
    if os.path.isdir(category_path): # Check if it's a directory
        label = 0 if "cat" in category else 1  # cat=0, dog=1
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            img_arr = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img_arr is not None:  # Check if the image was loaded successfully
                img_arr = cv2.resize(img_arr, (img_size, img_size))
                X.append(img_arr.flatten())  # flatten to 1D
                y.append(label)
            else:
                print(f"Warning: Could not read image {img_path}") # Print a warning if the image could not be read

X = np.array(X)
y = np.array(y)

print("Data shape:", X.shape, "Labels:", y.shape)

Data shape: (8005, 12288) Labels: (8005,)


# ***Split Data & Train SVM***

In [29]:
!ls /content/dataset/training_set/training_set

cats  dogs


In [28]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM with RBF kernel
svm = SVC(kernel='rbf', C=1, gamma='scale')
svm.fit(X_train, y_train)

# Evaluate
y_pred = svm.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.665209244222361


# ***Test on New Image***

In [30]:
def predict_image(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is not None:  # Check if the image was loaded successfully
        img = cv2.resize(img, (img_size, img_size))
        img_flat = img.flatten().reshape(1, -1)
        pred = svm.predict(img_flat)[0]
        return "Dog" if pred == 1 else "Cat"
    else:
        return f"Error: Could not read image from {path}"

# Example usage with a path from the test set
print(predict_image("/content/dataset/test_set/test_set/cats/cat.4001.jpg"))

Cat
